In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

In [3]:
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [5]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)


model = Net()

# Uncomment below for GPU
#model = model.cuda()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [6]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        
        # Uncomment below for GPU
        #data, target = data.cuda(), target.cuda()
        
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [7]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        
        # Uncomment below for GPU
        #data, target = data.cuda(), target.cuda()
        
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
for epoch in range(1, 10):
    train(epoch)
    test()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  app.launch_new_instance()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.308618
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.284532
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.281765
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.265394
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.258156
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.225708
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.208165
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.168379
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.120663
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.051949
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.896570
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.716296
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.527683
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.255475
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.111572
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.000798
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.816904
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.646747
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.788772
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.713736
Train Epoch: 1 [12800/60000 (

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  if __name__ == '__main__':
C:\Anaconda3\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  if sys.path[0] == '':



Test set: Average loss: 0.1797, Accuracy: 9479/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.118503
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.258159
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.157767
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.136873
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.189037
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.301847
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.087506
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.167024
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.363986
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.187793
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.330112
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.189293
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.195451
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.153266
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.166042
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.184624
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.207052
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.241384
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.123610
Train Epoch: 2 [12

Train Epoch: 3 [44160/60000 (74%)]	Loss: 0.152229
Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.052857
Train Epoch: 3 [45440/60000 (76%)]	Loss: 0.127949
Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.101818
Train Epoch: 3 [46720/60000 (78%)]	Loss: 0.056271
Train Epoch: 3 [47360/60000 (79%)]	Loss: 0.126800
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.096686
Train Epoch: 3 [48640/60000 (81%)]	Loss: 0.096003
Train Epoch: 3 [49280/60000 (82%)]	Loss: 0.089503
Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.121604
Train Epoch: 3 [50560/60000 (84%)]	Loss: 0.050842
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.087412
Train Epoch: 3 [51840/60000 (86%)]	Loss: 0.128016
Train Epoch: 3 [52480/60000 (87%)]	Loss: 0.115440
Train Epoch: 3 [53120/60000 (88%)]	Loss: 0.131907
Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.176085
Train Epoch: 3 [54400/60000 (91%)]	Loss: 0.150644
Train Epoch: 3 [55040/60000 (92%)]	Loss: 0.102099
Train Epoch: 3 [55680/60000 (93%)]	Loss: 0.164374
Train Epoch: 3 [56320/60000 (94%)]	Loss: 0.112267


Train Epoch: 5 [28800/60000 (48%)]	Loss: 0.017351
Train Epoch: 5 [29440/60000 (49%)]	Loss: 0.079075
Train Epoch: 5 [30080/60000 (50%)]	Loss: 0.106318
Train Epoch: 5 [30720/60000 (51%)]	Loss: 0.100018
Train Epoch: 5 [31360/60000 (52%)]	Loss: 0.201606
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.028618
Train Epoch: 5 [32640/60000 (54%)]	Loss: 0.156006
Train Epoch: 5 [33280/60000 (55%)]	Loss: 0.057725
Train Epoch: 5 [33920/60000 (57%)]	Loss: 0.064609
Train Epoch: 5 [34560/60000 (58%)]	Loss: 0.115171
Train Epoch: 5 [35200/60000 (59%)]	Loss: 0.076596
Train Epoch: 5 [35840/60000 (60%)]	Loss: 0.058987
Train Epoch: 5 [36480/60000 (61%)]	Loss: 0.056817
Train Epoch: 5 [37120/60000 (62%)]	Loss: 0.097156
Train Epoch: 5 [37760/60000 (63%)]	Loss: 0.040473
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.036793
Train Epoch: 5 [39040/60000 (65%)]	Loss: 0.178102
Train Epoch: 5 [39680/60000 (66%)]	Loss: 0.079207
Train Epoch: 5 [40320/60000 (67%)]	Loss: 0.044699
Train Epoch: 5 [40960/60000 (68%)]	Loss: 0.049727


Train Epoch: 7 [13440/60000 (22%)]	Loss: 0.037458
Train Epoch: 7 [14080/60000 (23%)]	Loss: 0.018321
Train Epoch: 7 [14720/60000 (25%)]	Loss: 0.029418
Train Epoch: 7 [15360/60000 (26%)]	Loss: 0.039929
Train Epoch: 7 [16000/60000 (27%)]	Loss: 0.020694
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.105735
Train Epoch: 7 [17280/60000 (29%)]	Loss: 0.028392
Train Epoch: 7 [17920/60000 (30%)]	Loss: 0.106984
Train Epoch: 7 [18560/60000 (31%)]	Loss: 0.074504
Train Epoch: 7 [19200/60000 (32%)]	Loss: 0.042349
Train Epoch: 7 [19840/60000 (33%)]	Loss: 0.013019
Train Epoch: 7 [20480/60000 (34%)]	Loss: 0.038340
Train Epoch: 7 [21120/60000 (35%)]	Loss: 0.071575
Train Epoch: 7 [21760/60000 (36%)]	Loss: 0.106625
Train Epoch: 7 [22400/60000 (37%)]	Loss: 0.077626
Train Epoch: 7 [23040/60000 (38%)]	Loss: 0.145557
Train Epoch: 7 [23680/60000 (39%)]	Loss: 0.007592
Train Epoch: 7 [24320/60000 (41%)]	Loss: 0.008202
Train Epoch: 7 [24960/60000 (42%)]	Loss: 0.139437
Train Epoch: 7 [25600/60000 (43%)]	Loss: 0.040416


Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.088355

Test set: Average loss: 0.0500, Accuracy: 9842/10000 (98%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.087341
Train Epoch: 9 [640/60000 (1%)]	Loss: 0.066545
Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.040999
Train Epoch: 9 [1920/60000 (3%)]	Loss: 0.099721
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.085640
Train Epoch: 9 [3200/60000 (5%)]	Loss: 0.050341
Train Epoch: 9 [3840/60000 (6%)]	Loss: 0.116850
Train Epoch: 9 [4480/60000 (7%)]	Loss: 0.005386
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.016129
Train Epoch: 9 [5760/60000 (10%)]	Loss: 0.080230
Train Epoch: 9 [6400/60000 (11%)]	Loss: 0.042818
Train Epoch: 9 [7040/60000 (12%)]	Loss: 0.071930
Train Epoch: 9 [7680/60000 (13%)]	Loss: 0.025063
Train Epoch: 9 [8320/60000 (14%)]	Loss: 0.013275
Train Epoch: 9 [8960/60000 (15%)]	Loss: 0.151167
Train Epoch: 9 [9600/60000 (16%)]	Loss: 0.186964
Train Epoch: 9 [10240/60000 (17%)]	Loss: 0.015989
Train Epoch: 9 [10880/60000 (18%)]	Loss: 0.094974
Train Epoch: 9 [11